In [1]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    os.chdir(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as th
import torch.nn as nn
from torch.nn import functional as F
from envs import BlendEnv

In [3]:
os.chdir(os.path.join(curr_dir, "decision-transformer\\gym\\"))
from decision_transformer.models.decision_transformer import DecisionTransformer
os.chdir(curr_dir)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_dict = th.load("./trained_dt.pth")

C:\Users\adame\AppData\Local\Temp\ipykernel_9304\3768740294.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = th.load("./trained_dt.pth")


In [5]:
import yaml, json
with open("./configs/30.yaml", "r") as f:
    s = "".join(f.readlines())
    cfg = yaml.load(s, Loader=yaml.FullLoader)
    
layout = "simple"

In [6]:
with open(f"./configs/json/connections_{layout}.json" ,"r") as f:
    connections_s = f.readline()
connections = json.loads(connections_s)

with open(f"./configs/json/action_sample_{layout}.json" ,"r") as f:
    action_sample_s = f.readline()
action_sample = json.loads(action_sample_s)

In [7]:
def get_sbp(connections):
    sources = list(connections["source_blend"].keys())
    
    b_list = list(connections["blend_blend"].keys())
    for b in connections["blend_blend"].keys():
        b_list += connections["blend_blend"][b]
    b_list += list(connections["blend_demand"].keys())
    blenders = list(set(b_list))
    
    p_list = []
    for p in connections["blend_demand"].keys():
        p_list += connections["blend_demand"][p]
    demands = list(set(p_list))
    
    return sources, blenders, demands
sources, blenders, demands = get_sbp(connections)

In [8]:
T = 6
if layout == "base":
    sigma = {"s1":{"q1": 0.06}, "s2":{"q1": 0.26}}
    sigma_ub = {"p1":{"q1": 0.16}, "p2":{"q1": 1}}
    sigma_lb = {"p1":{"q1": 0}, "p2":{"q1": 0}}
else:
    sigma = {s:{"q1": 0.06} for s in sources}
    sigma_ub = {d:{"q1": 0.16} for d in demands}
    sigma_lb = {d:{"q1": 0} for d in demands}
    
s_inv_lb = {s: 0 for s in sources}
s_inv_ub = {s: 999 for s in sources}
d_inv_lb = {d: 0 for d in demands}
d_inv_ub = {d: 999 for d in demands}
betaT_d = {d: 1 for d in demands} # Price of sold products
b_inv_ub = {j: 30 for j in blenders} 
b_inv_lb = {j: 0 for j in blenders}
betaT_s = {s: cfg["env"]["product_cost"]  for s in sources} # Cost of bought products

if cfg["env"]["uniform_data"]:
    if cfg["env"]["max_pen_violations"] < 999:
        max_ep_length = 50
        tau0   = {s: [np.random.normal(20, 3) for _ in range(max_ep_length)] for s in sources}
        delta0 = {d: [np.random.normal(20, 3) for _ in range(max_ep_length)] for d in demands}
        T = max_ep_length
        
    else:
        tau0   = {s: [np.random.normal(20, 3) for _ in range(13)] for s in sources}
        delta0 = {d: [np.random.normal(20, 3) for _ in range(13)] for d in demands}
else:
    tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
    delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [9]:
M,Q,P,B,Z,D = 0, 0, 0, 0, 1, 0
env = BlendEnv(v = True, 
               D = D, 
               Q = Q, 
               P = P, 
               B = B, 
               Z = Z, 
               M = M,
               reg = cfg["env"]["reg"],
               reg_lambda = cfg["env"]["reg_lambda"],
               MAXFLOW = cfg["env"]["maxflow"],
               alpha = cfg["env"]["alpha"],
               beta = cfg["env"]["beta"],
               connections = connections, 
               action_sample = action_sample,
               tau0 = tau0,delta0 = delta0,
               sigma = sigma,
               sigma_ub = sigma_ub, sigma_lb = sigma_lb,
               s_inv_lb = s_inv_lb, s_inv_ub = s_inv_ub,
               d_inv_lb = d_inv_lb, d_inv_ub = d_inv_ub,
               betaT_d = betaT_d, betaT_s = betaT_s,
               b_inv_ub = b_inv_ub,
               b_inv_lb = b_inv_lb)

In [10]:
model = DecisionTransformer(
            state_dim=env.observation_space.shape[0],
            act_dim=env.action_space.shape[0],
            max_length=20,
            max_ep_len=12,
            hidden_size=128,
            n_layer=3,
            n_head=1,
            n_inner=4*128,
            activation_function='relu',
            n_positions=1024,
            resid_pdrop=0.1,
            attn_pdrop=0.1
        )

In [11]:
model.load_state_dict(model_dict)

<All keys matched successfully>

In [38]:
import pickle
with open("C:/Users/adame/OneDrive/Bureau/CODE/BlendingRL/decision-transformer/gym/data/blend-medium-v4.pkl", "rb") as f:
    data = pickle.load(f)

In [43]:
data[0].keys()

dict_keys(['observations', 'next_observations', 'actions', 'rewards', 'dones'])

In [55]:
data[0]["observations"]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 25.9       ,  0.        ,
         0.        , 12.9       , 25.1       ,  0.        , 19.5       ,
        14.5       , 11.6       , 12.3       , 18.7       , 13.5       ,
         0.        , 21.7       , 16.        ,  0.        , 11.9       ,
         0.        , 23.5       ,  0.        ,  0.        , 13.4       ,
        20.6       ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        25.9       ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.26      , 12.9       , 25.1       ,  0.        ,
        19.5       , 14.5       , 11.6       , 12.3       , 18.7       ,
        13.5       ,  0.        , 21.7       , 16.        ,  0.        ,
        11.9       ,  0.        , 23.5       ,  0.        ,  0.        ,
        13.4    

In [58]:
th.Tensor(data[0]["rewards"]).unsqueeze(1)

tensor([[161.0483],
        [161.1483],
        [161.4483],
        [118.5483],
        [ 70.1483],
        [ 46.8483]])

In [61]:
model.forward(th.Tensor(data[0]["observations"]), th.Tensor(data[0]["actions"]), th.Tensor(data[0]["rewards"]), th.Tensor(data[0]["rewards"]).unsqueeze(1), th.LongTensor(np.arange(6)))

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 3 is not equal to len(dims) = 4